## Dog Breeds

https://www.kaggle.com/orangutan/keras-vgg19-starter

https://www.kaggle.com/gaborfodor/use-pretrained-keras-models-lb-0-3

    

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import pickle

import cv2                 # working with, mainly resizing, images
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import time
import matplotlib.pyplot as plt
% matplotlib inline

import time

In [2]:
os.chdir(r"D:\My Computer\DATA\Dog_Breed_Identification")

train_dir = "train"
test_dir = "test"
os.listdir()

['breed_class.csv',
 'labels.csv',
 'logs',
 'pickle',
 'sample_submission.csv',
 'VGG19.csv']

In [15]:
labels = pd.read_csv("labels.csv")
sample_submission= pd.read_csv("sample_submission.csv")
targets_series = pd.Series(labels['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

one_hot_labels = np.asarray(one_hot)

In [4]:
# ID has 32 characters
# File names have 36 characters
# suggesting the four last digits differentiate the dog within breeds

In [6]:
def dataprep():
    print("Dog Breed Number:", labels.shape)
    print("Training Size:", len(os.listdir(train_dir)))
    print("Test Size:", len(os.listdir(test_dir)))
    print("Sample Sub Size:", sample_submission.shape)

    im_size = 90

    x_train = []
    y_train = []
    x_test = []

    i = 0 
    for f, breed in tqdm(labels.values):
        img = cv2.imread('train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
        
    for f in tqdm(sample_submission['id'].values):
        img = cv2.imread('test/{}.jpg'.format(f))
        x_test.append(cv2.resize(img, (im_size, im_size)))
        
    for (x) in [(x_train, "x_train"), (y_train,"y_train"), (x_test, 'x_test')]:
        with open("pickle/{}.pickle".format(x[1]), 'wb') as f:
            pickle.dump(x[0], f)
        
#dataprep()

In [7]:
# Load
dic= {}
for name in ["x_train","y_train","x_test"]:
    open_file = open("./Pickle/{}.pickle".format(name), "rb")
    dic[name] = pickle.load(open_file)
    open_file.close()
    
y_train_raw = np.array(dic["y_train"], np.uint8)
x_train_raw = np.array(dic["x_train"], np.float32) / 255.
x_test  = np.array(dic["x_test"], np.float32) / 255.

del dic

num_class = y_train_raw.shape[1]
X_train, X_valid, Y_train, Y_valid = \
train_test_split(x_train_raw, y_train_raw, test_size=0.15, random_state=1)

print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

del x_train_raw, y_train_raw
    
## SEE ./ PICKLE FOLDER CREATOR

(10222, 90, 90, 3)
(10222, 120)
(10357, 90, 90, 3)


In [8]:
# Helpers
# Write
def write_model(model, modelname):
    preds = model.predict(x_test, verbose=0)
    sub = pd.DataFrame(preds)
    # Set column names to those generated by the one-hot encoding earlier
    col_names = one_hot.columns.values
    sub.columns = col_names
    # Insert the column id from the sample_submission at the start of the data frame
    sub.insert(0, 'id', sample_submission['id'])
    sub.to_csv("{}.csv".format(modelname), index=False)
       
im_size = 90
# Store Result, Parameters and Validation Accuracy

In [9]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

# Pretrained
from keras.applications.vgg19 import VGG19

Using TensorFlow backend.


In [10]:
# VGG19 Pretrained Model
# Not sure if I really need to iterate over this?
    
# Research Base Models
base_model = VGG19(weights='imagenet',
                   include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)

# This is the model we will train
# This outputs the softmax, probabilistic consideration
predictions = Dense(num_class, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 90, 90, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 90, 90, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 90, 90, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 45, 45, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 45, 45, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 45, 45, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 22, 22, 128)       0         
__________

In [11]:
# Play with Min_delta
ES = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=40,
          verbose=1, mode='auto')
# Figure out how to assign name to it
TB = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=2,  
          write_graph=True, write_images=False)
# Broken?
MC = keras.callbacks.ModelCheckpoint('./save', monitor='val_acc', verbose=1,
                             save_best_only=True, save_weights_only=True,
                             mode='auto', period=5)


callbacks_list = [ES,TB]

tensorboard --logdir=models:"D:\My Computer\DATA\Dog_Breed_Identification\logs"

In [13]:
import datetime
callbacks_list=[
                  keras.callbacks.TensorBoard(log_dir="./logs/{}".format(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H_%M_%S')),
                        histogram_freq=0, write_graph=False, write_images=False),
                  keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=.01, patience=7,
                        verbose=0, mode='auto')]

start = time.time()
model.fit(X_train, Y_train,
          validation_data=(X_valid, Y_valid),
          verbose=2, callbacks=callbacks_list,
          epochs=200)

end = time.time()
print("Model took %0.2f seconds to train"%(end - start))

Train on 8688 samples, validate on 1534 samples
Epoch 1/200
15s - loss: 4.5413 - acc: 0.0540 - val_loss: 4.3019 - val_acc: 0.0815
Epoch 2/200
14s - loss: 3.5751 - acc: 0.1967 - val_loss: 4.1188 - val_acc: 0.1121
Epoch 3/200
14s - loss: 3.0978 - acc: 0.2868 - val_loss: 4.0655 - val_acc: 0.1258
Epoch 4/200
14s - loss: 2.7433 - acc: 0.3750 - val_loss: 4.0499 - val_acc: 0.1258
Epoch 5/200
14s - loss: 2.4620 - acc: 0.4442 - val_loss: 4.0183 - val_acc: 0.1310
Epoch 6/200
14s - loss: 2.2287 - acc: 0.4964 - val_loss: 4.0728 - val_acc: 0.1219
Epoch 7/200
14s - loss: 2.0366 - acc: 0.5517 - val_loss: 4.1162 - val_acc: 0.1252
Epoch 8/200
14s - loss: 1.8562 - acc: 0.6019 - val_loss: 4.0840 - val_acc: 0.1408
Epoch 9/200
14s - loss: 1.7054 - acc: 0.6469 - val_loss: 4.1350 - val_acc: 0.1375
Epoch 10/200
14s - loss: 1.5646 - acc: 0.6910 - val_loss: 4.1706 - val_acc: 0.1239
Epoch 11/200
14s - loss: 1.4542 - acc: 0.7167 - val_loss: 4.1663 - val_acc: 0.1460
Epoch 12/200
14s - loss: 1.3353 - acc: 0.7513 - 

In [16]:
modelname = "VGG19"
write_model(model, modelname)

In [ ]:
## Model from Scratch